In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [53]:
df = pd.read_csv("../data/skylab_instagram_datathon_dataset.csv", sep=";")

df.shape

(704313, 15)

In [54]:
# information about the data before processing it -> run again after processing
df.describe(include='all')

,period,period_end_date,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,calculation_type,followers,pictures,videos,comments,likes
count,704313,704313,704313,704313,704313,676558,458589,676558,458589,704313,5.792580e+05,695803.000000,684349.000000,6.953430e+05,6.959770e+05
unique,1,455,20,54,706,423,26,401,30,1,NaN,NaN,NaN,NaN,NaN
top,Weekly,2023-07-08,Luxury & Premium & Mainstream,US Softlines Analyst Interest List,All Brands,LVMH Moet Hennessy Louis Vuitton SE,United States of America,LVMH Moet Hennessy Louis Vuitton SE,New York Stock Exchange,Metric Value,NaN,NaN,NaN,NaN,NaN
freq,704313,1635,216241,113744,27755,27576,193509,27576,132071,704313,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.342432e+07,344.272540,61.244426,3.043246e+04,2.163189e+06
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.106975e+08,2777.396873,564.468480,2.766459e+05,1.803193e+07
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.013372e+05,19.000000,2.000000,3.710000e+02,1.727700e+04
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.104144e+06,44.000000,7.000000,1.435000e+03,9.318400e+04
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.479325e+06,122.000000,20.000000,5.872000e+03,4.577470e+05


# Process data

## Data cleaing

### Replace

In [55]:
df = df.map(lambda x: x.strip().lower() if isinstance(x, str) else x)

In [56]:
# Converting the 'period_end_of_week' column to datetime
df['period_end_date'] = pd.to_datetime(df['period_end_date'])

In [57]:
# REPLACE DOMICILE_COUNTRY_NAME

# replace the china;hong kong with hong kong
df['domicile_country_name'] = df['domicile_country_name'].replace('china;hong kong', 'hong kong')

# remove the sign ";" in column domicile_country_name
df['domicile_country_name'] = df['domicile_country_name'].str.replace(';', '')

# if empty, fill with nan
df['domicile_country_name'] = df['domicile_country_name'].replace('', np.nan)

In [ ]:
# REPLACE DOMICILE_COUNTRY_NAME

# replace the china;hong kong with hong kong
df['domicile_country_name'] = df['domicile_country_name'].replace('china;hong kong', 'hong kong')

# remove the sign ";" in column domicile_country_name
df['domicile_country_name'] = df['domicile_country_name'].str.replace(';', '')

# if empty, fill with nan
df['domicile_country_name'] = df['domicile_country_name'].replace('', np.nan)

In [ ]:
# REPLACE PRIMARY_EXCHANGE_NAME

# remove the sign ";" in column domicile_country_name
df['primary_exchange_name'] = df['primary_exchange_name'].str.replace(';', '')

# if empty, fill with nan
df['primary_exchange_name'] = df['primary_exchange_name'].replace('', np.nan)

### Remove

In [58]:
# Remove "All_Brands" and "Don't Use it"
df = df[df["business_entity_doing_business_as_name"] != "All Brands"]
# TODO: Remove "Don't Use it"

In [59]:
# remove unnecessary columns
# Here: 'period', 'calculation_type'
df = df.drop(columns=['period', 'calculation_type'])

In [60]:
# certain entries have exact data except of "compset"
# we want to take the union of all of them

grouping_columns = [col for col in df.columns if col != 'compset']

df.fillna('Group_Null', inplace=True)
result = df.groupby(grouping_columns).agg({'compset': lambda x: set(x)}).reset_index()
df = result
df.replace('Group_Null', np.nan, inplace=True)


# df.describe(include='all')

# to test
#result[(result["business_entity_doing_business_as_name"] == "24S") & (result["period_end_date"] == "2017-05-13")]
#result[(result["business_entity_doing_business_as_name"] == "dd's Discounts") & (result["period_end_date"] == "2023-09-09")]
#result[(result["business_entity_doing_business_as_name"] == "dd's Discounts") & (result["period_end_date"] == "2023-09-16")]

/tmp/ipykernel_14304/591243391.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Group_Null' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('Group_Null', inplace=True)
/tmp/ipykernel_14304/591243391.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('Group_Null', np.nan, inplace=True)


In [61]:
result.shape

(325732, 13)

## Add features

In [62]:
# split the date into year, month, day
df['Year'] = df['period_end_date'].dt.year
df['Month'] = df['period_end_date'].dt.month
df['Day'] = df['period_end_date'].dt.day

In [63]:
# get the difference between the current date and the previous date
df = df.sort_values(by=['business_entity_doing_business_as_name', 'period_end_date'])
df['date_diff_prev'] = df['period_end_date'].diff().dt.days
df['date_diff_prev'] = df['date_diff_prev'].fillna(7)

In [64]:
#df[df['date_diff_prev'] == 0]

In [66]:
df.describe(include='all')

,period_end_date,compset_group,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,followers,pictures,videos,comments,likes,compset,Year,Month,Day,date_diff_prev
count,325732,325732,325732,298040,198093,298040,198433,2.615950e+05,321826.000000,315947.000000,3.216080e+05,3.219380e+05,325732,325732.000000,325732.000000,325732.000000,325732.000000
unique,NaN,20,706,423,22,401,30,NaN,NaN,NaN,NaN,NaN,116,NaN,NaN,NaN,NaN
top,NaN,apparel retail,all brands,lvmh moet hennessy louis vuitton se,united states of america,lvmh moet hennessy louis vuitton se,new york stock exchange,NaN,NaN,NaN,NaN,NaN,{beauty & boutique},NaN,NaN,NaN,NaN
freq,NaN,74801,27692,9632,86036,9632,58937,NaN,NaN,NaN,NaN,NaN,44266,NaN,NaN,NaN,NaN
mean,2019-07-22 20:01:03.306030336,NaN,NaN,NaN,NaN,NaN,NaN,2.352977e+07,606.507936,110.170576,5.212107e+04,3.717640e+06,NaN,2019.065695,6.413806,15.668399,0.007156
min,2015-01-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,NaN,2015.000000,1.000000,1.000000,-3178.000000
25%,2017-06-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,2.277570e+05,18.000000,2.000000,3.030000e+02,1.125400e+04,NaN,2017.000000,3.000000,8.000000,7.000000
50%,2019-08-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,9.433140e+05,46.000000,8.000000,1.362000e+03,7.055350e+04,NaN,2019.000000,6.000000,16.000000,7.000000
75%,2021-09-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,4.225668e+06,143.000000,26.000000,6.480000e+03,4.031650e+05,NaN,2021.000000,9.000000,23.000000,7.000000
max,2023-09-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,3.502565e+09,141746.000000,35905.000000,1.732046e+07,7.120711e+08,NaN,2023.000000,12.000000,31.000000,7.000000
